In [1]:
# Imports

import pandas as pd
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import the data

train = pd.read_csv('train.csv', encoding='UTF-8')

In [3]:
train.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [4]:
train.isnull().sum()

tweet_id     0
tweet        1
sentiment    0
dtype: int64

In [5]:
train.dropna(inplace=True)

In [6]:
train.isnull().sum()

tweet_id     0
tweet        0
sentiment    0
dtype: int64

In [7]:
# Delete ItemID

#train.drop('ItemID', axis=1, inplace=True)

In [8]:
train.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [9]:
import re
from string import punctuation

In [10]:
train['tweet']

0       #sxswnui #sxsw #apple defining language of tou...
1       Learning ab Google doodles! All doodles should...
2       one of the most in-your-face ex. of stealing t...
3       This iPhone #SXSW app would b pretty awesome i...
4       Line outside the Apple store in Austin waiting...
                              ...                        
7269    @mention Google plze Tammi.  I'm in middle of ...
7270    RT @mention ÷¼ Are you all set? ÷_ {link} ÷...
7271    RT @mention Aha! Found proof of lactation room...
7272    We just launched our iPad app at #SXSW! Get al...
7273    The next fin serv battle is vs Apple, GOOG, Mo...
Name: tweet, Length: 7273, dtype: object

In [11]:
all_sentiments1 = '\n'.join(train['tweet'])

In [12]:
all_sentiments1

"#sxswnui #sxsw #apple defining language of touch with different dialects becoming smaller\nLearning ab Google doodles! All doodles should be light, funny &amp; innovative, with exceptions for significant occasions. #GoogleDoodle #sxsw\none of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At #SXSW, Apple schools the mkt experts&quot;  {link}\nThis iPhone #SXSW app would b pretty awesome if it didn't crash every 10mins during extended browsing. #Fuckit #Illmakeitwork\nLine outside the Apple store in Austin waiting for the new iPad #SXSW  {link}\n#technews One lone dude awaits iPad 2 at Apple\x89Ûªs SXSW store {link} #Tech_News #Apple #iPad_2 #SXSW #tablets #tech\nSXSW Tips, Prince, NPR Videos, Toy Shopping With Zuckerberg.\r\n{link}  #sxsw  #ipad\nNU user RT @mention New #UberSocial for #iPhone now in the App Store includes UberGuide to #SXSW sponsored by #Mashable\nFree #SXSW sampler on iTunes {link} #FreeMusic\nI think I might go all weekend without seeing th

In [13]:
def remove_hash(text):
    hash_pat = r'#'
    return re.sub(hash_pat, ' ', text)
def remove_mention(text):
    mention_pat = r'@mention|@[a-z]+'
    return re.sub(mention_pat, ' ', text, flags=re.I)
def remove_short_link(text):
    short_link_pat = r"bit\.ly/[a-z0-9/\-:\.=%;,\+\*())&\$!@\[\]#\?~_\.']*"
    return re.sub(short_link_pat, ' ', text, flags=re.I) 
def remove_http_link(text):
    link_permit = r"[a-z0-9/\-:\.=%;,\+\*())&\$!@\[\]#\?~_\.']"
    http_link_pat = r"http[s]?://"+link_permit+"+|//"+link_permit+"+|[\w\.]+\.[a-z]+/"+link_permit+"+" 
    return re.sub(http_link_pat, ' ', text, flags=re.I)
def remove_sub_link(text):
    link_pat = r'{link}'
    return re.sub(link_pat, ' ', text, flags=re.I)
def remove_html_char(text):
    html_char_pat = r'&[a-z]+;'
    return re.sub(html_char_pat, ' ', text, flags=re.I)    
def remove_date(text):
    pipe = r'|'
    date_pat_mon = str()
    months =   [r'January',
                r'February',
                r'March',
                r'April',
                r'May',
                r'June',
                r'July',
                r'August',
                r'September',
                r'October',
                r'November',
                r'December']
    for month in months:
        date_pat_mon = date_pat_mon + month+r' \d\d, \d\d\d\d|'
        date_pat_mon = date_pat_mon + month[:3]+r' \d\d, \d\d\d\d|'
    date_pat_mon = date_pat_mon[:-1]
#     date_pat_mon
    date_pat =  r'\d\d/\d\d/\d\d\d\d|\d\d/\d\d/\d\d' + pipe + \
                r'\d\d\.\d\d\.\d\d\d\d|\d\d\.\d\d\.\d\d' + pipe + \
                r'\d\d-\d\d-\d\d\d\d|\d\d-\d\d-\d\d' + pipe + \
                r'{}'.format(date_pat_mon)
    date_pat = r'{}'.format(date_pat)
    return re.sub(date_pat, ' ', text, flags=re.I)
def remove_short_date(text):
    pipe = r'|'
    short_date_pat = r'[\d]?\d/\d\d[\d\d]?' + pipe + r'[\d]?\d\.\d\d[\d\d]?'
    short_date_pat = r'{}'.format(short_date_pat)
    return re.sub(short_date_pat, ' ', text)
def remove_time(text):
    pipe = r'|'
    time_pat =  r'\d\d:\d\d:\d\d[ ]?pm'+pipe+\
                r'\d\d:\d\d:\d\d[ ]?am'+pipe+\
                r'\d\d:\d\d:\d\d'+pipe+\
                r'\d\d:\d\d:\d\d'+pipe+\
                r'[\d]?\d:\d\d[ ]?pm'+pipe+\
                r'[\d]?\d:\d\d[ ]?am'+pipe+\
                r'[\d]?\d:\d\d'+pipe+\
                r'[\d]?\d:\d\d'+pipe+\
                r'[\d]?\d.\d\d[ ]?pm'+pipe+\
                r'[\d]?\d.\d\d[ ]?am'+pipe+\
                r'[\d]?\d.\d\d'+pipe+\
                r'[\d]?\d.\d\d'
    time_pat= r'{}'.format(time_pat)
    return re.sub(time_pat,' ', text, flags=re.I)

def remove_sxsw(text):
    sxsw_pat = r"sxsw[a-z]*"
    return re.sub(sxsw_pat, ' ', text, flags=re.I) 

def remove_austin_texas(text):
    austin_pat = r"austin"
    texas_pat = r"texas|tx"
    temp = text
    temp = re.sub(austin_pat, ' ', temp, flags=re.I) 
    temp = re.sub(texas_pat, ' ', temp, flags=re.I) 
    return temp
def remove_punctuation(text):
    punctuation_pat_s = r'\'s'
    punctuation_pat_t = r'\'t'
    punctuation_pat_d = r'\'d'
    punctuation_pat_ve = r'\'ve'
    punctuation_pat_ll = r'\'ll'
    temp = text
    temp = re.sub(punctuation_pat_s, ' ', temp, flags=re.I)
    temp = re.sub(punctuation_pat_t, 't', temp, flags=re.I)
    temp = re.sub(punctuation_pat_d, ' would', temp, flags=re.I)
    temp = re.sub(punctuation_pat_ve, ' have', temp, flags=re.I)
    temp = re.sub(punctuation_pat_ll, ' will', temp, flags=re.I)
    return temp
def remove_rt(text):
    rt_pat = r'RT'
    return re.sub(rt_pat, ' ', text)


def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return re.sub(emoji_pattern,'', text)
def remove_not_alnum(text):
    """
    remove everything that is not alpha numeric
    """
    only_text = r'[^a-z0-9 ]'
    return re.sub(only_text, ' ', text, flags=re.I)  



In [14]:
def clean_text(text):
    #replace links witn {link}
    text = remove_short_link(text)
    text = remove_http_link(text)
    #remove {link}
    text = remove_sub_link(text)
    #hash
    text = remove_hash(text)
    #mention
    text = remove_mention(text)
    #rt
    text = remove_rt(text)
    #html spl chars
    text = remove_html_char(text)
    #time
    text = remove_time(text)
    #svxm
    text=remove_sxsw(text)
    #texas
    text=remove_austin_texas(text)
    #date
    text = remove_date(text)
    text = remove_short_date(text)
    #punctuation
    text = remove_punctuation(text)
    #emotjis
    text = remove_emojis(text)
    #remove spl chars
    text = remove_not_alnum(text)
    #return lower
    return text.lower()

In [15]:
train['tweet'] = train['tweet'].apply(lambda x:clean_text(x))

In [16]:
train.isnull().sum()

tweet_id     0
tweet        0
sentiment    0
dtype: int64

In [17]:
all_sentiments = '\n'.join(train['tweet'])

In [18]:
all_sentiments

'       apple defining language of touch with different dialects becoming smaller\nlearning ab google doodles  all doodles should be light  funny   innovative  with exceptions for significant occasions   googledoodle   \none of the most in your face ex  of stealing the show in yrs      at     apple schools the mkt experts    \nthis iphone    app would b pretty awesome if it didnt crash every 10mins during extended browsing   fuckit  illmakeitwork\nline outside the apple store in   waiting for the new ipad      \n technews one lone dude awaits ipad 2 at apple   s   store    tech news  apple  ipad 2     tablets  tech\n  tips  prince  npr videos  toy shopping with zuckerberg           ipad\nnu user     new  ubersocial for  iphone now in the app store includes uberguide to    sponsored by  mashable\nfree    sampler on itunes    freemusic\ni think i might go all weekend without seeing the same ipad case twice      \n    official    app      go       android  iphone  ipad\nit  official  i m 

In [19]:
all_sentiments[:50]

'       apple defining language of touch with diffe'

In [20]:
#nltk.download('stopwords')

In [21]:
#from nltk.corpus import stopwords
import nltk

from string import punctuation
from nltk.stem.porter import *
punc = list(punctuation)

# tokenize
train['tokenized_text'] = [nltk.word_tokenize(x) for x in train['tweet']]


In [22]:
#train['tokenized_text'] = train['tokenized_text'].apply(lambda row: [word for word in row if word not in punc])

train['tokenized_text'] = train['tokenized_text'].apply(lambda row: [word for word in row if word not in punc])

#train['tweet'] = train['tweet'].apply(lambda row: [word for word in row if word not in punc])

In [23]:
#allsentiment='\n'.join(train['tokenized_text'])

In [24]:
stemmer.stem('battery')

NameError: name 'stemmer' is not defined

In [25]:
stemmer = PorterStemmer()

train['tokenized_text'] = train['tokenized_text'].apply(lambda x: [stemmer.stem(i) for i in x])
train['tokenized_text'] = train['tokenized_text'].apply(lambda x: ' '.join(x))


In [ ]:
train['tokenized_text']

In [26]:
X = train['tokenized_text']
y = train['sentiment']

In [27]:

from gensim.models import Word2Vec

tokenized_tweet =train['tokenized_text'].apply(lambda x: x.split()) 

model_w2v = Word2Vec(tokenized_tweet, size=5000,  window=5)

In [28]:
model_w2v.wv.most_similar("store")

[('up', 0.9983832836151123),
 ('popup', 0.9980398416519165),
 ('pop', 0.9979608654975891),
 ('open', 0.9963197708129883),
 ('appl', 0.9949199557304382),
 ('temporari', 0.9943603873252869),
 ('downtown', 0.9939084053039551),
 ('temp', 0.9928567409515381),
 ('line', 0.9923226833343506),
 ('shop', 0.9856176376342773)]

In [ ]:
tokenized_tweet

In [ ]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model_w2v.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [ ]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn import metrics

from sklearn.metrics import classification_report
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
X = train['tokenized_text']
y = train['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=10)

In [ ]:
X_train.isnull().sum()


In [ ]:
y_train.isnull().sum()

In [ ]:
tokenized_tweet_xtrain =X_train.apply(lambda x: x.split()) 

In [ ]:
tokenized_tweet_xtrain.isnull().sum()

In [ ]:
tokenized_tweet_xtrain

In [ ]:
y_train.isnull().sum()

In [ ]:
num_features = 300
trainDataVecsxtrain = getAvgFeatureVecs(tokenized_tweet_xtrain, model_w2v, num_features)

In [ ]:
trainDataVecsxtrain.dtype

In [ ]:

trainDataVecsxtrain = pd.DataFrame(trainDataVecsxtrain).fillna(0)

In [ ]:
y_train.value_counts()

In [ ]:
Tn1=3428/0.4
#print(tn)

In [ ]:
ratio_3 = {0:int(round(Tn1*.25)),1:int(round(Tn1*.40)),2:int(round(Tn1*.35))} #new ratio dict
ratio_3
print(ratio_3)
sum(ratio_3.values())

In [ ]:
from imblearn.over_sampling import SMOTE


sm1 = SMOTE(random_state=42,sampling_strategy=ratio_3)
X_trainsm, y_trainsm = sm1.fit_resample(trainDataVecsxtrain, y_train)

In [ ]:
y_trainsm.value_counts()

In [ ]:
tokenized_tweet_xtest =X_test.apply(lambda x: x.split()) 



In [ ]:
num_features = 300
trainDataVecsxtest = getAvgFeatureVecs(tokenized_tweet_xtest, model_w2v, num_features)

In [ ]:
trainDataVecsxtest = pd.DataFrame(trainDataVecsxtest).fillna(0)

In [ ]:
# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
#lr= LogisticRegression()
print("Fitting random forest to training data....")    
forest = forest.fit(X_trainsm, y_trainsm)

y_pred = forest.predict(trainDataVecsxtest)
#print(f'CLF report for {name}')
score = metrics.f1_score(y_test, y_pred,average='weighted')
print(score)
print(classification_report(y_test, y_pred))

In [ ]:
rf = RandomForestClassifier()
lr = LogisticRegression()
gb = GradientBoostingClassifier()
xgb= XGBClassifier()

In [ ]:
print("Fitting xgboost to training data....")    
xgbmodel = xgb.fit(X_trainsm, y_trainsm)

In [ ]:
y_pred = xgbmodel.predict(trainDataVecsxtest)
#print(f'CLF report for {name}')
score = metrics.f1_score(y_test, y_pred,average='weighted')
print(score)
print(classification_report(y_test, y_pred))

In [ ]:
lrmodel = lr.fit(X_trainsm, y_trainsm)

In [ ]:
y_predlr = lrmodel.predict(trainDataVecsxtest)
#print(f'CLF report for {name}')
score = metrics.f1_score(y_test, y_predlr,average='weighted')
print(score)
print(classification_report(y_test, y_predlr))

In [ ]:
test = pd.read_csv('test.csv', encoding='utf-8')

In [ ]:
test['tweet'] = test['tweet'].apply(lambda x:clean_text(x))

In [ ]:
test['tokenized_text'] = [nltk.word_tokenize(x) for x in test['tweet']]

In [ ]:
stemmer = PorterStemmer()

train['tokenized_text'] = train['tokenized_text'].apply(lambda x: [stemmer.stem(i) for i in x])

test['tokenized_text'] = test['tokenized_text'].apply(lambda x: ' '.join(x))


In [ ]:
tokenized_tweet_test =test['tokenized_text'].apply(lambda x: x.split()) 

In [ ]:
testDataVecs = getAvgFeatureVecs(tokenized_tweet_test,  model_w2v, num_features)

In [ ]:
result = xgbmodel.predict(testDataVecs)
output = pd.DataFrame(data={"id":test["tweet_id"], "sentiment":result})
output.to_csv( "submissionxgb.csv", index=False, quoting=3 )

In [ ]:

'

In [ ]:
#trainDataVecs.isnull().sum()

In [ ]:
#pd.train['sentiment'].isnull()

In [ ]:
test.head()

In [ ]:
#tok = TreebankWordTokenizer()

In [ ]:
# Find unique tokens

#unique_tokens = len(set(tok.tokenize(all_sentiments)))

#print(f'Unique unprocessed tokens - {unique_tokens}')

In [ ]:
# Unique lower cased tokens

#unique_lower_case = [t.lower() for t in tok.tokenize(all_sentiments)]
#unique_lc_count = len(set(unique_lower_case))

#print(f'Unique lower case unprocessed tokens - {unique_lc_count}')

In [ ]:
#ps = PorterStemmer()

In [ ]:
# Unique lower cased stemmed tokens

#unique_lc_stemmed = [ps.stem(t) for t in unique_lower_case]
#unique_lc_stem_count = len(set(unique_lc_stemmed))

#print(f'Unique lower case stemmed tokens - {unique_lc_stem_count}')

In [ ]:

# Load stop words

#stw = stopwords.words('english')

In [ ]:
# Unique lower cased stemmed w/o stop wordstokens

#unique_lc_stem_wo_stpwrds = [t for t in unique_lc_stemmed if t not in stw]
#unique_lc_stem_wo_stw_count = len(set(unique_lc_stem_wo_stpwrds))

#print(f'Unique lower case stemmed tokens without stop words - {unique_lc_stem_wo_stw_count}')

In [ ]:
# Create a word cloud

#wc = WordCloud(stopwords=stw, background_color='white', max_words=500).generate(all_sentiments.lower())

In [ ]:
#%matplotlib inline
#plt.figure(figsize=(15, 10))
#plt.clf()
#plt.imshow(wc)
#plt.axis('off')
#plt.show()

## Sentiment Analysis

In [ ]:
# Create X and Y
#train['tokenized_text'] = [nltk.word_tokenize(x) for x in train['tokenized_text']]

#X = train['tokenized_text']


In [ ]:
#from imblearn.over_sampling import SMOTE


In [ ]:
#sm=SMOTE(sampling_strategy=0.25,random_state=10)

In [ ]:
#X_res, y_res = sm.fit_resample(X, y)

In [ ]:
# Split data in to train and test



In [ ]:
#cv = CountVectorizer()

In [ ]:
#X_train_vec = cv.fit_transform(X_train)

In [ ]:
#from imblearn.over_sampling import RandomOverSampler


In [ ]:
# Creating count vectorizer from training data



In [ ]:
# Instantiate all the models



In [ ]:
# Convert my test data into vectors for prediction

#X_test_vec = cv.transform(X_test)

In [ ]:
# Train the models and check CLF report





In [ ]:
# Train the models and check CLF report for TFIDF vectorizer



## Cleaning with RE

**List of unwanted items**
1. Hashtags (#)
2. Mentions (@)
3. Links (http://, https://)
4. Short link (t.co, bit.ly)
5. Numbers
6. Emojis
7. Emoticons
8. Punctuations